In [4]:
import os

comments_directory = '../preprocessed_comments/'
comments_files = [f for f in os.listdir(comments_directory) if f.endswith('liked.csv')]
original_directory = '../data2/'
original_files = [f for f in os.listdir(original_directory) if f.endswith('.json')]
titles = [f.split('.')[0] for f in original_files]
original_files

["Fox News-Watch: President Biden's State of the Union address and the GOP's response.json",
 'CNN-Full Speech: President Biden’s 2024 State of the Union address.json',
 'MSNBC-WATCH: Full 2024 State of the Union address | MSNBC.json']

In [5]:
import os

import google.generativeai as genai
import warnings
warnings.filterwarnings("ignore")

GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

from google.generativeai.types import HarmCategory, HarmBlockThreshold

def target_stance_detection(text: str, video_title: str):
    model = genai.GenerativeModel('gemini-pro')
    # message = "The following text is a comment on the political news video titled: " + video_title + "\nThe description of this video is:" + description + "\nIdentify the stance of the following comment towards this subject: " + text + "\nChoose from Conservative, Liberal, Other"
    message = "This news video titled " + video_title + ". Analyze the following comment and identify the political stance (liberal, conservative, etc.) in one word: " + text + "\nChoose from Conservative, Liberal, Other"
    response = model.generate_content(message, safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
        }
    )
    stance = response.text
    print(text + ": " + stance)
    return stance


target_stance_detection("This is an outrageous attack on our freedoms! We need to hold the government accountable!", "CNN-Full Speech: President Biden’s 2024 State of the Union address")

This is an outrageous attack on our freedoms! We need to hold the government accountable!: Conservative


'Conservative'

In [6]:
import json
import pandas as pd
import os


liked_comments_files = [f for f in os.listdir(comments_directory) if f.endswith('liked.csv')]

for file in liked_comments_files:
    df = pd.read_csv(comments_directory + file)
    video_title = file.split('.')[0].removesuffix('_liked')
    df['stance'] = df['comment'].apply(lambda x: target_stance_detection(x, video_title))
    df.to_csv(comments_directory + file, index=False)

look at all those criminals: Conservative
cnn turned their comments off edit i just realized everyone did seconds after i commented let the people speak: Conservative
this wasnt a state of the union it was a state of confusion: Conservative
im 62 and have heard quite a few state of the union address speeches tonight i heard literally one lie right after another despicable disgraceful sickening are a few words coming to mind: Conservative
the look on johnsons face shows the true state of our nation its sad that we have a president that cares about everyone but us: Conservative
one of the worst state of the union speeches ever it was full of lies and hate 405: Conservative
president biden should consider running for the presidency of ukraine as his initial focus on the topic highlights his approach to governance prioritizing foreign affairs particularly concerning ukraine demonstrates his commitment and leadership despite the promising reports on the economy from economists the benefits 

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).